# Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
# disable tensorflow log level infos
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # show only errors

import sys
import pandas as pd

if '../../../../notebooks/' not in sys.path:
    sys.path.append('../../../../notebooks/')
if 'src' not in sys.path:
    sys.path.insert(0, 'src')

import utils.constants as cts

from data_loaders.data_loader import DLName
from gt_loaders.gt_names import GTName
from exp_runner import ExperimentRunner
from base.benchmark_dataset import BenchmarkDataset
from base.model_evaluator import DataSource, DataPredSelection
from base.base_models import BaseModel
from base.optimizers import Optimizer
from m_utils.mtl_approach import MTLApproach
from m_utils.nas_mtl_approach import NAS_MTLApproach
from m_utils.constants import ICAO_REQ, MNIST_TASK

 ==> Restrict GPU memory growth: True


# Start Network runner

In [3]:
kwargs = { 
    'use_neptune': True,
    'exp_params' : {
        'name': 'neural_arch_search',
        'description': 'NAS Approach 2 with FVC-ICAO dataset with 3 trials and patience',
        'tags': ['ground_truth', 'nas', 'nas_approach_2', 'fvc'],
        'src_files': ["src"]
    },
    'properties': {
        'approach': NAS_MTLApproach.APPROACH_2,
        'benchmarking': {
            'use_benchmark_data': False,
            'benchmark_dataset': BenchmarkDataset.MNIST,
            'tasks': list(MNIST_TASK)
        },
        'icao_data': {
            'icao_gt': {
                'use_gt_data': True,
                'gt_names': {
                    'train_validation': [],
                    'test': [],
                    'train_validation_test': [GTName.FVC]
                },
            },
            'icao_dl': {
                'use_dl_data': False,
                'tagger_model': None
            },
            'reqs': list(ICAO_REQ),
            'aligned': False
        },
        'balance_input_data': False,
        'train_model': True,
        'save_trained_model': True,
        'exec_nas': True,
        'orig_model_experiment_id': '',
        'sample_training_data': False,
        'sample_prop': 1.0
    },
    'net_train_params': {
        'base_model': BaseModel.MOBILENET_V2,
        'batch_size': 32,
        'n_epochs': 50,
        'early_stopping': 5,
        'learning_rate': 1e-3,
        'optimizer': Optimizer.ADAMAX,
        'dropout': 0.3
    },
    'nas_params': {
        'max_blocks_per_branch': 5,
        'controller_epochs': 50,
        'controller_batch_size': 32,
        'n_trials': 3
    }
}

runner = ExperimentRunner(**kwargs)

-------------------- Init ExperimentRunner -------------------
---------------------------
Parent Process ID: 93209
Process ID: 106076
---------------------------
-----
Use Neptune:  True
-----
-------------------
Args: 
{'exp_params': {'description': 'NAS Approach 2 with FVC-ICAO dataset with 3 '
                               'trials and patience',
                'name': 'neural_arch_search',
                'src_files': ['src'],
                'tags': ['ground_truth', 'nas', 'nas_approach_2', 'fvc']},
 'nas_params': {'controller_batch_size': 32,
                'controller_epochs': 50,
                'max_blocks_per_branch': 5,
                'n_trials': 3},
 'net_train_params': {'base_model': <BaseModel.MOBILENET_V2: {'name': 'mobilnet_v2', 'target_size': (224, 224), 'prep_function': <function preprocess_input at 0x7f0e2cfda700>}>,
                      'batch_size': 32,
                      'dropout': 0.3,
                      'early_stopping': 5,
                      'lear

/home/guilherme/anaconda3/envs/mteval-icao-reqs/submodules/icao_nets_training/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


# Load Data

In [4]:
runner.load_training_data()

-------------------- load training data -------------------
Loading data
Loading GT FVC - TRAIN split...
..Ignoring 0 empty label values
Input data.shape: (4928, 26)
Loading GT FVC - VALIDATION split...
..Ignoring 0 empty label values
Input data.shape: (547, 26)
Loading GT FVC - TEST split...
..Ignoring 0 empty label values
Input data.shape: (288, 26)
Data loaded


In [5]:
runner.train_data.head()

,origin,img_name,mouth,rotation,l_away,eyes_closed,close,hat,dark_glasses,frames_heavy,...,sh_head,blurred,ink_mark,skin_tone,washed_out,pixelation,hair_eyes,background,red_eyes,aligned
0,fvc,/home/guilherme/data1/Dropbox/Link to Desktop/...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,False
1,fvc,/home/guilherme/data1/Dropbox/Link to Desktop/...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,False
2,fvc,/home/guilherme/data1/Dropbox/Link to Desktop/...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,False
3,fvc,/home/guilherme/data1/Dropbox/Link to Desktop/...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,False
4,fvc,/home/guilherme/data1/Dropbox/Link to Desktop/...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,False


# <font color='red'>Producing Fake Data</font>

# Data Generators

In [6]:
runner.setup_data_generators()

-------------------- setup data generators -------------------
Starting data generators
Found 4928 validated image filenames.
Found 547 validated image filenames.
Found 288 validated image filenames.
TOTAL: 5763

Logging class indices
 .. MTL model not logging class indices!

Logging class labels
 COMPLIANT label: 1
 NON_COMPLIANT label: 0
 DUMMY label: -1
 DUMMY_CLS label: 2
 NO_ANSWER label: -99


# Setup Experiment

In [7]:
runner.setup_experiment()

-------------------- create experiment -------------------
Setup neptune properties and parameters
Properties and parameters setup done!


# Labels Distribution

In [8]:
%%capture
runner.summary_labels_dist()

# Neural Architecture Search

In [ ]:
runner.run_neural_architeture_search()

-------------------- run neural architecture search -------------------
Executing neural architectural search
  Memory reseted

==================== STARTING NEW TRIAL ====================
 selecting new config...
  Memory is empty
 controller_pred: [[0.39429787 0.148866   0.24217187 0.2146643 ]]


 ------ Training 1 | Config: {'n_denses_0': 2, 'n_denses_1': 1, 'n_denses_2': 2, 'n_denses_3': 2} -----

Creating model...
Model created
Training MOBILENET_V2 network
 .. Not fine tuning base model...
  .. Total params: 4,233,966
  .. Trainable params: 1,975,982
  .. Non-trainable params: 2,257,984


# Create Model with Best Config Found

In [ ]:
runner.create_model()

# Visualize Model

In [ ]:
runner.visualize_model(outfile_path=f"figs/nas/nas_model_approach_2.png")

In [ ]:
%%capture
runner.model_summary()

# Training Model

In [ ]:
runner.train_model()

# Plots

In [ ]:
runner.draw_training_history()

# Load Best Model

In [ ]:
runner.load_best_model()

# Saving Trained Model

In [ ]:
runner.save_model()

# Test Trained Model

## Validation Split

In [ ]:
runner.set_model_evaluator_data_src(DataSource.VALIDATION)
runner.test_model(verbose=False)

## Test Split

In [ ]:
runner.set_model_evaluator_data_src(DataSource.TEST)
runner.test_model(verbose=False)

# Visualize Model Classification

# Finishing Experiment Manager

In [ ]:
runner.finish_experiment()

# Testing Network Modification

# Test - Customized Loss Function

# Test - 

In [ ]:
import tensorflow as tf
import numpy as np

import os
# disable tensorflow log level infos
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # show only errors

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


def __create_rnn_model():
    model = Sequential([
        Dense(4, activation="relu"),
        Dense(64, activation="relu"),
        Dense(4, activation="softmax")
    ])

    model.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy"])

    return model

def __preprocess_config(config):
    return np.linalg.norm(config)
    


np.random.seed(2)

X = np.random.rand(400,4)
y = np.random.rand(400,4)

# X = tf.expand_dims(X, axis=0)
# y = np.expand_dims(y, axis=0)

print(X.shape)
print(y.shape)

X_test = np.random.rand(20,4)
y_test = np.random.rand(20,4)

# X_test = tf.expand_dims(X_test, axis=0)
# y_test = tf.expand_dims(y_test, axis=0)

m = __create_rnn_model()

m.fit(X,y, batch_size=32, epochs=5)

loss, acc = m.evaluate(X_test,y_test, batch_size=32)

print(f'loss: {loss}%')
print(f'acc: {round(acc*100,2)}%')

print(f'prediction: {m.predict(np.array(X_test[0]).reshape(1,4))}')

In [ ]:
import sys
import random

from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model
from tensorflow.keras.layers import Add, Concatenate, Embedding, LSTM, LSTMCell, RNN, Reshape
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import losses, metrics
from tensorflow.keras.models import load_model
from tensorflow.keras.initializers import RandomUniform, HeNormal, GlorotNormal

import tensorflow as tf


def get_weight_initializer(initializer=None, seed=None):
    if initializer is None:
        return HeNormal()
    elif initializer == "lstm":
        return RandomUniform(minval=-0.1, maxval=0.1)
    else:
        return GlorotNormal()


def get_weight_regularizer(regularizer=None, rate=1e-4):
    if regularizer is None:
        return regularizers.l2(rate)
    else:
        return regularizer(rate)


class ControllerRNNController(object):
    def __init__(self,
                 controller_network_name,
                 num_nodes,
                 num_opers,
                 input_x,
                 reward=0,
                 temperature=5.0,
                 tanh_constant=2.5,
                 model_file=None,
                 lstm_cell_units=32,
                 baseline_decay=0.999,
                 opt=Adam(learning_rate=0.00035, decay=1e-3, amsgrad=True)):

        self.controller_network_name = controller_network_name
        self.num_nodes = num_nodes
        self.num_opers = num_opers
        self.reward = reward
        self.input_x = input_x
        self.temperature = temperature
        self.tanh_constant = tanh_constant
        self.lstm_cell_units = lstm_cell_units
        self.opt = opt
        self.model_file = model_file

        self.controller_rnn = self.generate_controller_rnn()
        self.baseline = None
        self.baseline_decay = baseline_decay

        #self.graph = tf.get_default_graph()

    def lstm_reshape(self,
                     inputs,
                     name_prefix,
                     index,
                     reshaped_inputs=None,
                     initial=False):
        name_prefix = "{0}_{1}_{2}".format(self.controller_network_name,
                                           name_prefix, index)
        cell = LSTMCell(
            self.lstm_cell_units,
            kernel_initializer=get_weight_initializer(initializer="lstm"),
            recurrent_initializer=get_weight_initializer(initializer="lstm"))
        if initial:
            x = RNN(
                cell,
                return_state=True,
                name="{0}_{1}".format(name_prefix, "lstm"))(inputs)
        else:
            x = RNN(
                cell,
                return_state=True,
                name="{0}_{1}".format(name_prefix, "lstm"))(
                    reshaped_inputs, initial_state=inputs[1:])
        rx = Reshape(
            (-1, self.lstm_cell_units),
            name="{0}_{1}".format(name_prefix, "reshape"))(x[0])
        return x, rx

    def dense_softmax(self, inputs, num_classes, name_prefix, index):
        name_prefix = "{0}_{1}_{2}".format(self.controller_network_name,
                                           name_prefix, index)
        y = Dense(
            num_classes, name="{0}_{1}".format(name_prefix, "dense"))(inputs)
        y = Activation(
            activation="softmax",
            name="{0}_{1}".format(name_prefix, "softmax"))(y)
        return y

    def generate_controller_rnn(self):
        outputs = []
        controller_input = Input(shape=(1, 1,), name="{0}_{1}".format(self.controller_network_name, "input"))

        for i in range(2, self.num_nodes):
            for o in ["inputL", "inputR", "operL", "operR"]:
                if i == 2 and o == "inputL":
                    _x, _rx, _initial = controller_input, None, True
                else:
                    _x, _rx, _initial = x, rx, False

                if o in ["inputL", "inputR"]:
                    _num_classes = i
                else:
                    _num_classes = self.num_opers

                x, rx = self.lstm_reshape(
                    inputs=_x,
                    name_prefix=o,
                    index=i,
                    reshaped_inputs=_rx,
                    initial=_initial)
                y = self.dense_softmax(
                    inputs=x[0],
                    num_classes=_num_classes,
                    name_prefix=o,
                    index=i)
                outputs.append(y)

        controller_rnn = Model(inputs=controller_input, outputs=outputs)

        if self.model_file is not None and os.path.exists(self.model_file):
            controller_rnn.load_weights(self.model_file)
        return controller_rnn

    def compile_controller_rnn(self):
        def _controller_loss(y_true, y_pred):
            if self.baseline is None:
                self.baseline = 0
            else:
                self.baseline -= (1 - self.baseline_decay) * (self.baseline - self.reward)
            return y_pred * (self.reward - self.baseline)

        def _define_loss(controller_loss):
            outputs_loss = {}
            for i in range(2, self.num_nodes):
                outputs_loss["{0}_{1}_{2}_{3}".format(self.controller_network_name, "inputL", i, "softmax")] = controller_loss
                outputs_loss["{0}_{1}_{2}_{3}".format(self.controller_network_name, "inputR", i, "softmax")] = controller_loss
                outputs_loss["{0}_{1}_{2}_{3}".format(self.controller_network_name, "operL", i, "softmax")] = controller_loss
                outputs_loss["{0}_{1}_{2}_{3}".format(self.controller_network_name, "operR", i, "softmax")] = controller_loss
            return outputs_loss

        self.controller_rnn.compile(loss=_define_loss(_controller_loss), optimizer=self.opt)

    def save_model(self):
        self.controller_rnn.save_weights(self.model_file)

    def train_controller_rnn(self,
                             targets,
                             batch_size=1,
                             epochs=50,
                             callbacks=[EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')]):
        #with self.graph.as_default():
        self.compile_controller_rnn()
        self.controller_rnn.fit(
            self.input_x,
            targets,
            epochs=epochs,
            batch_size=batch_size,
            verbose=0)

    def softmax_predict(self):
        #with self.graph.as_default():
        self.compile_controller_rnn()
        return self.controller_rnn.predict(self.input_x)

    def random_sample_softmax(self, controller_pred):
        sample_softmax = []
        for cp in controller_pred:
            cp /= self.temperature
            cp = self.tanh_constant * np.tanh(cp)
            cp = np.exp(cp) / np.sum(np.exp(cp))
            cp = np.array([np.random.multinomial(1, cp[0])])
            sample_softmax.append(cp)
        return sample_softmax

    def convert_pred_to_cell(self, controller_pred):
        cell_pred = {}
        for p in range(2, self.num_nodes):
            pos = list(range((p - 2) * 4, ((p - 2) * 4) + 4))
            cell_pred[p] = {
                "L": {
                    "input_layer": np.argmax(controller_pred[pos[0]]),
                    "oper_id": np.argmax(controller_pred[pos[2]])
                },
                "R": {
                    "input_layer": np.argmax(controller_pred[pos[1]]),
                    "oper_id": np.argmax(controller_pred[pos[3]])
                }
            }
        return cell_pred

    def convert_pred_to_ydict(self, controller_pred):
        ydict = {}
        name_prefix = self.controller_network_name
        for i in range(2, self.num_nodes):
            pos = list(range((i - 2) * 4, ((i - 2) * 4) + 4))
            ydict["{0}_{1}_{2}_{3}".format(name_prefix, "inputL", i, "softmax")] = controller_pred[pos[0]]
            ydict["{0}_{1}_{2}_{3}".format(name_prefix, "inputR", i, "softmax")] = controller_pred[pos[1]]
            ydict["{0}_{1}_{2}_{3}".format(name_prefix, "operL", i, "softmax")] = controller_pred[pos[2]]
            ydict["{0}_{1}_{2}_{3}".format(name_prefix, "operR", i, "softmax")] = controller_pred[pos[3]]
        return ydict


In [ ]:
contr = ControllerRNNController("netname", num_nodes=3, num_opers=3, input_x=X)

In [ ]:
from tensorflow.keras.utils import plot_model

#contr.controller_rnn.summary()
plot_model(contr.controller_rnn, expand_nested=True, show_shapes=True)

# Test LSTM

In [ ]:
import tensorflow as tf

import os
# disable tensorflow log level infos
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # show only errors

inputs = tf.random.normal([32, 10, 8])
lstm = tf.keras.layers.LSTM(4, return_sequences=True, return_state=True)

whole_seq_output, final_memory_state, final_carry_state = lstm(inputs)

print(whole_seq_output.shape, final_memory_state.shape, final_carry_state.shape)